# Introducción al Pre-training de LLMs

El pre-training de los modelos de lenguaje es una técnica clave que permite entrenar modelos a gran escala en corpus de texto no supervisado. Este enfoque ha revolucionado el procesamiento del lenguaje natural, ya que los modelos entrenados aprenden representaciones generales del lenguaje antes de ser ajustados (fine-tuning) para tareas específicas como clasificación de texto, traducción o respuestas automáticas.

En este Colab, cubriremos los principios básicos del pre-training de LLMs, los métodos más comunes, y un ejemplo práctico de pre-entrenamiento en un conjunto de datos de texto.


# Empezamos con el pre-training

In [2]:
# Instalación de las bibliotecas necesarias
!pip install -U transformers datasets --quiet

Pare el pre-entrenamiento necesitamos un dataset, ya sea que el dataset sea local o importado de alguna librería es necesaria la muestra de datos para que el proceso

In [1]:
from datasets import load_dataset

# Cargar el dataset para pre-entrenamiento
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
print(dataset)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})


Notemos que este dataset esta dividido en tres grupos, test, train y validation. Así que no necesitamos dividir la muestra, pero les dejamos comentado una celda abajo sobre cómo se puede dividir un dataset.

Nuestra recomendación es dividir la muestra en dos grupos, un grupo para entrenar el modelo, y un segundo grupo para evaluar el desempeño del mismo. No queremos entrenar y evaluar con los mismos datos ya que eso puede generar que el modelo "memorice" las respuestas correctas en lugar de aprender realmente

In [3]:
"""
max_samples = 30000

# Limitamos el tamaño del dataset si es necesario
total_size = min(max_samples, len(dataset))
val_size = int(total_size * 0.2)
train_size = total_size - val_size

# División del conjunto de datos en entrenamiento y validación
split_ds = dataset.train_test_split(
    train_size=train_size,
    test_size=val_size,
    seed=42
)
"""

'\nmax_samples = 30000\n\n# Limitamos el tamaño del dataset si es necesario\ntotal_size = min(max_samples, len(dataset))\nval_size = int(total_size * 0.2)\ntrain_size = total_size - val_size\n\n# División del conjunto de datos en entrenamiento y validación\nsplit_ds = dataset.train_test_split(\n    train_size=train_size,\n    test_size=val_size,\n    seed=42\n)\n'

También requerimos de un modelo, en este caso usaremos GPT-2

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Definir el tokenizador y el modelo
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

# Asignar el token de padding como el eos_token (fin de secuencia)
tokenizer.pad_token = tokenizer.eos_token


Para utilizar las muestras del dataset, debemos pasarla por el tokenizer para que el modelo lo pueda procesar. Por lo que haremos una función que mapee las muestras en base al tokenizer

In [5]:
# Función para tokenizar los datos
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Tokenizamos los conjuntos de entrenamiento y validación por separado
tokenized_train = dataset['train'].map(tokenize_function, batched=True)
tokenized_val = dataset['test'].map(tokenize_function, batched=True)

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Ahora viene la parte del entramiento. Podemos importar un Trainer para configurarlo de manera sencilla. A modo de ejemplo:

In [6]:
from transformers import Trainer, TrainingArguments

# Configuración de los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

# Definir el trainer con los datos tokenizados
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train, # Usamos el conjunto de entrenamiento para entrenar
    eval_dataset=tokenized_val  # Usamos el conjunto de validación para evaluar
)


Una vez que configurarmos el training, podemos empezarlo

In [ ]:
# Entrenar el modelo
trainer.train() # Esto va a consumir toda la memoria RAM en el colab y se va a cortar


El proceso de entrenamiento siempre es muy costoso, por lo que entrenarlo todo de una vez necesita buenos recursos.

Sin embargo, no es el fin del mundo. Les presentamos un par de alternativas si el proceso es demasiado costoso:

1. Si estás utilizando un dataset grande, puedes reducir aún más el número de muestras (max_samples) o disminuir el batch size del Trainer. Disminuir el batch_size puede ayudar significativamente a reducir el uso de memoria.

2. La opción save_steps permite que el Trainer guarde el modelo periódicamente durante el entrenamiento, lo que es útil para evitar que el trabajo se pierda si se interrumpe el proceso. Puedes guardar el modelo y su estado cada ciertos pasos, como se muestra en el ejemplo anterior.

In [16]:
# Alternativa uno

training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,  # Puedes disminuir el número de épocas también
    per_device_train_batch_size=2,  # Disminuir batch size a 2
    save_steps=1000,  # Guardar más seguido para no perder el progreso
    save_total_limit=2,
    evaluation_strategy="steps",  # Evaluar cada ciertos pasos
    eval_steps=1000,  # Evaluar cada 1000 pasos
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train, # Usamos el conjunto de entrenamiento para entrenar
    eval_dataset=tokenized_val  # Usamos el conjunto de validación para evaluar
)

In [14]:
# Alternativa dos

training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=1000,  # Guardar el modelo cada 1000 pasos
    evaluation_strategy="steps",
    eval_steps=1000,  # Evaluar cada 1000 pasos
    save_total_limit=2,  # Mantener solo los últimos 2 checkpoints
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train, # Usamos el conjunto de entrenamiento para entrenar
    eval_dataset=tokenized_val  # Usamos el conjunto de validación para evaluar
)

Elige la alternativa que prefieras y comprobemos que pasa:

In [18]:
trainer.train()

ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.

Y podemos deternos luego del entramiento, guardando el modelo y el tokenizer para seguir siendo usados a futuro

In [ ]:
# Guardar el modelo pre-entrenado
model.save_pretrained("pretrained_model")
tokenizer.save_pretrained("pretrained_model")


Cuando queramos cargar el modelo para continuar el proceso es tan simple cómo esto

In [ ]:
trainer.train(resume_from_checkpoint=True)
